First, import needed modules

In [61]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [62]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [63]:
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_58981/690852991.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_58981/690852991.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_58981/690852991.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_csv(

View the data

In [64]:
btw09_es.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,Volksabstimmung,ZENTRUM,ADM,DKP,DIE VIOLETTEN,ödp,PIRATEN,RRP,Freie Union,Übrige
0,1,01,0,01,0,000,0,000025,0,1462,...,0,1,0,0,0,0,0,0,0,0
1,1,01,0,01,0,000,0,000026,0,1615,...,0,2,0,0,0,0,0,0,0,0
2,1,01,0,01,0,000,0,000027,0,1739,...,0,1,0,0,0,0,0,0,0,0
3,1,01,0,01,0,000,0,000028,0,1631,...,0,4,0,0,0,0,0,0,0,0
4,1,01,0,01,0,000,0,000029,0,1508,...,0,2,0,0,0,0,0,0,0,0


Add year column to dataframes

In [65]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es]
years = [2002, 2005, 2009, 2013, 2017]
for df, year in zip(dfs, years):
    df.insert(loc=len(df.columns), column='year', value=year)

In [66]:
btw05_es.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,MLPD,DIE PARTEI,CM,DSU,HP,HUMANWIRTSCHAFTSPARTEI,STATT Partei,UNABHÄNGIGE,Übrige,year
0,1,01,0,01,0,000,0,000001,0,1367,...,0,0,0,0,0,0,0,0,0,2005
1,1,01,0,01,0,000,0,000002,0,1393,...,0,0,0,0,0,0,0,0,0,2005
2,1,01,0,01,0,000,0,000003,0,1264,...,0,0,0,0,0,0,0,0,0,2005
3,1,01,0,01,0,000,0,000004,0,1195,...,0,0,0,0,0,0,0,0,0,2005
4,1,01,0,01,0,000,0,000005,0,1155,...,0,0,0,0,0,0,0,0,0,2005


Append dataframes

In [67]:
btw_es = pd.concat(dfs, sort=False)
btw_es.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,du,MG,MENSCHLICHE WELT,Gesundheitsforschung,V-Partei³,Bündnis C,DIE EINHEIT,MIETERPARTEI,Neue Liberale,Ungekürzte Wahlbezirksbezeichnung
0,1,01,0,01,0,000,1,0,1463,1272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,01,0,01,0,000,2,0,1245,1148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,01,0,01,0,000,3,0,1399,1298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,01,0,01,0,000,4,0,1031,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,01,0,01,0,000,5,0,1023,952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [68]:
btw_es['AGS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Gemeinde']
btw_es.head()
#dimensions = 443641 rows, 101 columns


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,MG,MENSCHLICHE WELT,Gesundheitsforschung,V-Partei³,Bündnis C,DIE EINHEIT,MIETERPARTEI,Neue Liberale,Ungekürzte Wahlbezirksbezeichnung,AGS
0,1,01,0,01,0,000,1,0,1463,1272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001000
1,1,01,0,01,0,000,2,0,1245,1148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001000
2,1,01,0,01,0,000,3,0,1399,1298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001000
3,1,01,0,01,0,000,4,0,1031,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001000
4,1,01,0,01,0,000,5,0,1023,952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01001000


Clean dataset

In [69]:
btw_es = btw_es.groupby(['AGS', 'year']).sum() # collapse data along AGS column, sums votes over all constituencies in a municipality 11348 columns left
btw_es.head()

Wahlkreis  Verbandsgemeinde  Bezirksart  Wahlberechtigte (A)  \
AGS      year                                                                 
01001000 2002         53                 0          35                65740   
         2005         53                 0          35                66970   
         2009         58                 0          60                69075   
         2013         56                 0          60                70321   
         2017         56                 0          60                70105   

               Wahlberechtigte ohne Sperrvermerk (A1)  \
AGS      year                                           
01001000 2002                                   57788   
         2005                                   58379   
         2009                                   60983   
         2013                                   60755   
         2017                                   57929   

               Wahlberechtigte mit Sperrvermerk (A2)  \
AGS      year                                          
01001000 2002                                   7949   
         2005                                   8591   
         2009                                   8092   
         2013                                   9566   
         2017                                  12176   

               Wahlberechtigte nach § 25 Abs. 2 BWO (A3)  Wähler (B)  \
AGS      year                                                          
01001000 2002                                        3.0       49054   
         2005                                        0.0       49002   
         2009                                        0.0       46583   
         2013                                        0.0       46827   
         2017                                        0.0       49694   

               Wähler mit Wahlschein (B1)  Ungültig  ...  Die Grauen   du  \
AGS      year                                        ...                    
01001000 2002                        7573     503.0  ...         0.0  0.0   
         2005                        8218     758.0  ...         0.0  0.0   
         2009                        7238    1204.0  ...         0.0  0.0   
         2013                        9084     589.0  ...         0.0  0.0   
         2017                       11528       0.0  ...         0.0  0.0   

                MG  MENSCHLICHE WELT  Gesundheitsforschung  V-Partei³  \
AGS      year                                                           
01001000 2002  0.0               0.0                   0.0        0.0   
         2005  0.0               0.0                   0.0        0.0   
         2009  0.0               0.0                   0.0        0.0   
         2013  0.0               0.0                   0.0        0.0   
         2017  0.0               0.0                   0.0        0.0   

               Bündnis C  DIE EINHEIT  MIETERPARTEI  Neue Liberale  
AGS      year                                                       
01001000 2002        0.0          0.0           0.0            0.0  
         2005        0.0          0.0           0.0            0.0  
         2009        0.0          0.0           0.0            0.0  
         2013        0.0          0.0           0.0            0.0  
         2017        0.0          0.0           0.0            0.0  

[5 rows x 92 columns]

Sum votes for other and Union parties

In [71]:
list(btw_es.columns)
andere = ['PDS',
 'REP',
 'GRAUE',
 'TIERSCHUTZ',
 'NPD',
 'ÖDP',
 'PBC',
 'FRAUEN',
 'BP',
 'FAMILIE',
 'CM',
 'BÜSO',
 'HP',
 'VIOLETTEN',
 'AUFBRUCH',
 'ZENTRUM',
 'KPD',
 'PRG',
 'SCHILL',
 'DEUTSCHLAND',
 'DKP',
 'DSU',
 'FP D',
 'ÜBRIGE',
 'Die Linke',
 'Offensive D',
 'Die Tierschutzpartei',
 'DIE FRAUEN',
 'BüSo',
 'Deutschland',
 'AGFG',
 'APPD',
 'MLPD',
 'DIE PARTEI',
 'HUMANWIRTSCHAFTSPARTEI',
 'STATT Partei',
 'UNABHÄNGIGE',
 'Übrige',
 'DIE LINKE',
 'Volksabstimmung',
 'ADM',
 'DIE VIOLETTEN',
 'ödp',
 'PIRATEN',
 'RRP',
 'Freie Union',
 'Tierschutzpartei',
 'Bündnis 21/RRP',
 'RENTNER',
 'BIG',
 'pro Deutschland',
 'FREIE WÄHLER',
 'PARTEI DER VERNUNFT',
 'Die PARTEI',
 'Bergpartei',
 'BGD',
 'NEIN',
 'WGr/EB',
 'PDV',
 'SGP',
 'DIE RECHTE',
 'Tierschutzallianz',
 'B*',
 'Die Grauen',
 'du',
 'MG',
 'MENSCHLICHE WELT',
 'Gesundheitsforschung',
 'V-Partei³',
 'Bündnis C',
 'DIE EINHEIT',
 'MIETERPARTEI',
 'Neue Liberale']
btw_es['Andere'] = btw_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_es['Union'] = btw_es[union].sum(axis=1) # sum votes for 'Union' Parties
btw_es = btw_es[['Wahlberechtigte (A)', 'Wähler (B)', 'Union', 'SPD', 'FDP', 'DIE LINKE', 'GRÜNE', 'AfD', 'Andere']]
btw_es.head()

Wahlberechtigte (A)  Wähler (B)  Union    SPD   FDP  DIE LINKE  \
AGS      year                                                                   
01001000 2002                65740       49054  16418  26309  2038        0.0   
         2005                66970       49002  16954  24261  1601        0.0   
         2009                69075       46583  13859  16552  4253     4697.0   
         2013                70321       46827  15647  18915   807     3250.0   
         2017                70105       49694  15140  15175  3421     5316.0   

               GRÜNE     AfD  Andere  
AGS      year                         
01001000 2002   2833     0.0   953.0  
         2005   2729     0.0  2699.0  
         2009   5520     0.0  5195.0  
         2013   4533  1459.0  4877.0  
         2017   5958  3441.0  6074.0

Merge with treatment dataset

In [72]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})
data = btw_es.join(treatment.set_index(['AGS']))
#data = pd.merge(btw_es, treatment, on='AGS', indicator=True)
#data = data.set_index('AGS', 'year')
#data = data.drop(columns=['_merge'])
data.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'Wähler (B)': 'Wähler'}, inplace = True)
data = data.reindex(columns=['GEN', 'Wahlberechtigte', 'Wähler', 'Union', 'SPD', 'FDP', 'DIE LINKE', 'GRÜNE', 'AfD', 'Andere', 'treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100'])
data.head()

GEN  Wahlberechtigte  Wähler  Union    SPD   FDP  \
AGS      year                                                           
01001000 2002  Flensburg            65740   49054  16418  26309  2038   
         2005  Flensburg            66970   49002  16954  24261  1601   
         2009  Flensburg            69075   46583  13859  16552  4253   
         2013  Flensburg            70321   46827  15647  18915   807   
         2017  Flensburg            70105   49694  15140  15175  3421   

               DIE LINKE  GRÜNE     AfD  Andere  treated_0  treated_10  \
AGS      year                                                            
01001000 2002        0.0   2833     0.0   953.0        0.0         0.0   
         2005        0.0   2729     0.0  2699.0        0.0         0.0   
         2009     4697.0   5520     0.0  5195.0        0.0         0.0   
         2013     3250.0   4533  1459.0  4877.0        0.0         0.0   
         2017     5316.0   5958  3441.0  6074.0        0.0         0.0   

               treated_20  treated_30  treated_50  treated_100  
AGS      year                                                   
01001000 2002         0.0         0.0         0.0          0.0  
         2005         0.0         0.0         0.0          0.0  
         2009         0.0         0.0         0.0          0.0  
         2013         0.0         0.0         0.0          0.0  
         2017         0.0         0.0         0.0          0.0